# Store Reddis Posts in the Database

This notebook uses pushshift.io APIs to store Reddis posts from
migraine subreddit in the database.

Accessing Reddis data via APIs is slow.  For example, pushshift.io has limits on how often given endpoint can be accessed.  In order to gather 10000 posts from pushshift.io it takes close to 8 hours since we needed to throttle our APIs to avoid `429 Too Many Requests` errors.  For this reason we chose to store data retrieved from pushshift.io in MongoDB.  MongoDB's document data model supports JSON and it has great query language.  Since pushshift.io already returns posts as JSON objects MongoDB is a great fit.

# MongoDB Setup

Rather than dealing with differences of MongoDB setup on different operating systems we decided to use Docker based approach.  Here is the `docker-compose.yml`:

```
version: "3"
services:
  mongo:
    image: mongo:latest
    volumes:
      - ./dbdata:/data/db
    ports:
      - 37017:27017

```

You can start mongodb using `docker-compose up -d`.  Notice Mongodb folder `/data/db` is default location for database files and `volumes` entry maps it to host's `/dbdata`.  You can change this if you want to use different folder on your system.  Mapping to the host's folder allows restarts of the container without losing the data.  In addition, we can just zip `dbdata` folder to share database files.


# Connect to Database

- Make sure you started mongodb with `docker-compose up -d`
- mongodb running in the container is exposed on port 37017 to prevent confusion in case you already have mongo installed on your system and running on default port 27017.

In [2]:
import requests
import time
import pprint
from pymongo import MongoClient

In [3]:
database_name = 'reddit-migraine'

client = MongoClient('mongodb://localhost:37017')
db = client[database_name]

# Create Indexes

You need to create indexes on the fields below so you can quickly access data when it grows big.


In [4]:
posts_collection = 'posts'

db[posts_collection].create_index('id', unique=True)
db[posts_collection].create_index('selftext', unique=False)
db[posts_collection].create_index('title', unique=False)
db[posts_collection].create_index('author', unique=False)

'author_1'

# API Helper Methods

These methods access Reddit posts and comments from pushshift.io server.

In [5]:
def get_posts(pushshift_url, subreddit_name, before_time, max_size=100):
    should_retry = True
    while should_retry:
        try:
            req = requests.get(f'{pushshift_url}/?subreddit={subreddit_name}&sort=desc&sort_type=created_utc&before={before_time}&size={max_entries}')
            output = req.json()
            should_retry = False
        except:
            print(f'retrying post...')
            time.sleep(5)
            should_retry = True
    return output

def get_comments(pushshift_url, comment_id, max_size=100):
    should_retry = True
    while should_retry:
        try:
            req = requests.get(f'{pushshift_url}/?link_id={comment_id}&limit={max_size}')
            output = req.json()
            should_retry = False
        except:
            print(f'retrying comment {comment_id}...')
            time.sleep(5)
            should_retry = True
    return output

# URLs and Constants

Define needed constants.

In [6]:
pushshift_post_url = 'https://api.pushshift.io/reddit/search/submission'
pushshift_comment_url = 'https://api.pushshift.io/reddit/comment/search'
subreddit_name = 'migraine'
max_entries = 10

# Get Posts

Get posts and comments associated with each post.  For each post store the comments in new field `comments` as a mongodb embedded array.

To start getting comments use `before_time` with current time as shown in the code below.  This will start with latest post and continue until it gets specified number of iterations or reads all of the available date from pushshift.io.

Notice that each iteration retrieves `max_entries` as defined above in constants.

After each iteration new `before_time` is printed out.  This is useful in case the code or website crashes and you want to start over.  This serves as checkpoint and you can take that new value and assign it to `before_time` variable and restart the cell below.

In [ ]:
before_time = int(time.time())  # current epoch time
total_posts = 0

def get_page_of_posts(before_time):
    posts = get_posts(pushshift_post_url, subreddit_name, before_time, max_entries)
    data = posts.get('data', [])

    for entry in data:
        comment_id = entry['id']
        comments = get_comments(pushshift_comment_url, comment_id)
        entry['comments'] = comments['data']
        time.sleep(1)
    return data

for _ in range(10000):
    posts = get_page_of_posts(before_time)
    for post in posts:
        db['posts'].insert_one(post)
    total_posts += len(posts)
    print(f'Inserted {len(posts)} posts... Total posts so far: {total_posts}')
    print(f'Last before time: {before_time}')
    before_time = posts[len(posts) - 1]['created_utc']
    print(f'Next before time: {before_time}')
    time.sleep(1)

print(f'Done.  Total new posts: {total_posts}')